In [1]:
import numpy as np


# 1D

In [208]:
out_c = 1
in_c = 3
kernel = 3
batch = 1
in_size = 5
out_size = ((in_size - kernel) // 1) + 1
A = np.arange(batch * in_c * in_size).reshape(batch, in_c, in_size)
A = np.array([[[1, 0, -1, 2, 1], [0, 1, 2, 1, 0], [1, 2, 0, -1, 0]]])
print(A)
# W = np.random.normal(0, 1.0, (out_c, in_c, kernel))
W = np.array([[[1, 0, 2], [2, 1, 1], [0, 1, 0]]])
B = np.random.normal(0, 1.0, (out_c))
# B = np.zeros((out_c))

print(A.shape)
print(W.shape)
print(B.shape)


[[[ 1  0 -1  2  1]
  [ 0  1  2  1  0]
  [ 1  2  0 -1  0]]]
(1, 3, 5)
(1, 3, 3)
(1,)


In [209]:
print(W)

[[[1 0 2]
  [2 1 1]
  [0 1 0]]]


In [210]:
# W_z = np.repeat(W[np.newaxis, :, :, :], batch, axis=0)
# B = np.repeat(B[np.newaxis, :], batch, axis=0)
# Z = np.zeros((batch, out_c, out_size), dtype=np.float)
# for l in range(out_c):
#     for i in range(in_size - kernel + 1):
#         Z[:, l, i] = np.sum(A[:, :, i:i+kernel] * W_z[:, l, :, :], axis=(1, 2))
#         Z[:, l, i] += B[:, l]
# print(Z)
# print(Z.shape)

Z = np.array([[[2, 1, 1]]])

In [211]:
dLdZ = Z
dZ_map = np.repeat(dLdZ[:, np.newaxis, :, :], in_c, axis=1)
print(dZ_map)
dLdW = np.zeros_like(W)  # TODO
for l in range(out_c):
    for i in range(kernel):
        # print(A[:, :, i : i + kernel].shape)
        # print(dZ_map[:, :, l, :].shape)
        # print(np.sum(A[:, :, i : i + out_size] * dZ_map[:, :, l, :], axis=(0, 2)))
        dLdW[l, :, i] = np.sum(A[:, :, i : i + out_size] * dZ_map[:, :, l, :], axis=(0, 2))

print(dLdW)

[[[[2 1 1]]

  [[2 1 1]]

  [[2 1 1]]]]
[[[ 1  1  1]
  [ 3  5  5]
  [ 4  3 -1]]]


In [ ]:
dLdA = np.zeros_like(A, dtype=np.float)  # TODO
dZ_map_padded = np.pad(dZ_map, ((0, 0), (0, 0), (0, 0), (kernel - 1, kernel - 1)), mode="constant")
print(dZ_map_padded)
W = np.flip(W, axis=(2))
W = np.repeat(W[np.newaxis, :, :, :], batch, axis=0)
print(W)
for l in range(out_c):
    for i in range(in_size):
        print("__________________________")
        print(f"Zmap [:, :, {l}, {i}:{i+kernel}]")
        print(dZ_map_padded[:, :, l, i : i + kernel])
        print(f"W [:, {l}, :, :]")
        print(W[:, l, :, :])
        # print(W[:, l, :, :].shape)
        # print(dZ_map[:, :, l, i : i + kernel].shape)
        # print(dLdA[:, :, i].shape)
        print("SUM")
        print(dZ_map_padded[:, :, l, i : i + kernel] * W[:, l, :, :])
        dLdA[:, :, i] += np.sum(dZ_map_padded[:, :, l, i : i + kernel] * W[:, l, :, :], axis=(2))
        print(dLdA)
        print("\n\n")

dLdA

# 2D


## Forwards

In [ ]:
out_c = 1
in_c = 3
kernel = 3
batch = 1
in_size = 5
out_size = ((in_size - kernel) // 1) + 1
A = np.arange(batch * in_c * in_size).reshape(batch, in_c, in_size)
A = np.array([[
    [[1, 1, -2, 2, -3], [2, -3, 1, 2, 1], [-1, 1, 1, -3, 2], [0, -2, -1, 0, 2], [-1, -3, -3, 1, 2]], 
    [[0, -1, -2, -3, 1], [-3, -1, -3, 1, 2], [-1, -2, 2, 2, -1], [2, -2, 1, 1, 1], [-1, -1, -3, -2, -2]]]])
print(A)
# W = np.random.normal(0, 1.0, (out_c, in_c, kernel))
W = np.array([
    [[[0, 0, -1], [-2, 0, 0], [-2, 1, 0]], [[-2, -2, -1], [1, -1, -1], [-2, 0, 1]]], 
    [[[-2, 1, 1], [1, -2, 1], [1, -1, -2]], [[-1, -1, -2], [-2, 0, -1], [-1, -2, 0]]], 
    [[[1,1, 1], [1, -1, -1], [0, 0, 1]], [[-1, -1, -1], [0, 1, 0], [0, -1, -1]]]
    ])
print(W)
# B = np.random.normal(0, 1.0, (out_c))
b = np.array([[2], [-1], [1]])
print(b)

print(A.shape)
print(W.shape)
print(B.shape)


In [39]:
output_w = (input_w - kernel_size) + 1
output_h = (input_h - kernel_size) + 1

Z = np.zeros((batch, out_channels, output_w, output_h), dtype=A.dtype)
W = np.repeat(W[np.newaxis, :, :, :, :], batch, axis=0)
B = np.repeat(b[np.newaxis, :], batch, axis=0)
print("A = ", A.shape)
print("W = ", W.shape)
print("B = ", B.shape)
for l in range(out_channels):
    for i in range(input_w - kernel_size + 1):
        for j in range(input_h - kernel_size + 1):
            Z[:, l, i, j] = (
                np.sum(A[:, :, i : i + kernel_size, j : j + kernel_size] * W[:, l, :, :, :], axis=(1, 2, 3)) + B[:, l]
            )

print(Z)

A =  (1, 1, 5, 5)
W =  (1, 1, 1, 3, 3)
B =  (1, 3, 1)
[[[[12 -6 10]
   [ 5  8  9]
   [-4 12 -7]]]]


### Backprop

In [ ]:
dLdZ = np.array([[[[1, 0, 1], [1, 2, 3], [0, 1, 2]]]])
# print(dLdZ)
# print(dLdZ.shape)
A = np.array([[[[0, 2, -3, 2, -3], [-2, -2, -2, -1, -2], [-3, -3, 2, -2, 1], [-3, -2, 1, -3, 1], [0, -1, 0, 2, -3]]]])
# print(A)
# print(A.shape)
W = np.array([[[[-1, 0, -2], [1, 1, -1], [-1, -1, 0]]]])
# print(W)
# print(W.shape)

batch = A.shape[0]
out_channels, in_channels, kernel_size, _ = W.shape
input_w, input_h = A.shape[2:]

dZ_map = np.repeat(dLdZ[:, np.newaxis, :, :, :], in_channels, axis=1)

# dLdb = np.sum(dLdZ, axis=(0, 2, 3))  # TODO
# print(dLdb)

dLdW = np.zeros_like(W, dtype=np.float)  # TODO
for l in range(out_channels):
    for i in range(kernel_size):
        for j in range(kernel_size):
            print("_" * 20)
            print(A[:, :, i : i + output_w, j : j + output_h])
            print(dZ_map[:, :, l, :, :])
            print(np.sum(
                A[:, :, i : i + output_w, j : j + output_h] * dZ_map[:, :, l, :, :], axis=(0, 2, 3)
            ))
            dLdW[l, :, i, j] = np.sum(
                A[:, :, i : i + output_w, j : j + output_h] * dZ_map[:, :, l, :, :], axis=(0, 2, 3)
            )
            print(dLdW)
print(dLdW)

# dLdA = np.zeros_like(A, dtype=np.float)  # TODO
# dZ_map_pad = np.pad(dZ_map, ((0, 0), (0, 0), (0, 0), (kernel_size - 1, kernel_size - 1), (kernel_size - 1, kernel_size - 1)), mode="constant")
# print(dZ_map_pad)
# W = np.flip(W, axis=(2, 3))
# print(W)
# W = np.repeat(W[np.newaxis, :, :, :, :], batch, axis=0)
# for l in range(out_channels):
#     for i in range(input_w):
#         for j in range(input_h):
#             print("_" * 20)
#             print(W[:, l, :, :, :])
#             print(dZ_map_pad[:, :, l, i : i + kernel_size, j : j + kernel_size])
#             print("Sum = ", np.sum(
#                 dZ_map_pad[:, :, l, i : i + kernel_size, j : j + kernel_size] * W[:, l, :, :, :], axis=(2, 3)
#             ))
#             dLdA[:, :, i, j] += np.sum(
#                 dZ_map_pad[:, :, l, i : i + kernel_size, j : j + kernel_size] * W[:, l, :, :, :], axis=(2, 3)
#             )
# print(dLdA)

# Flatten

In [58]:
x = np.array([[[[2, 0, 1], [1, 5, 1], [0, 1 ,0]], [[2, 5, 1], [1, 1, 1], [0, 1 ,0]]]])
y = x.flatten()
z = y.reshape(x.shape)
print(np.unravel_index(np.argmax(x), x.shape))
print(y)
print(z)

(0, 0, 1, 1)
[2 0 1 1 5 1 0 1 0 2 5 1 1 1 1 0 1 0]
[[[[2 0 1]
   [1 5 1]
   [0 1 0]]

  [[2 5 1]
   [1 1 1]
   [0 1 0]]]]


# Pooling

In [232]:
A = np.array([
    [
        [[2, 1, 2, -2, 1], [1, -2, 2, 3, 1], [3, -2, 1, 0, 0], [-2, -1, -2, 2, -1], [-1, 0, 3, 2, -1]]
        ]
    ])
print(A)
print(A.shape)

batch = A.shape[0]
out_channels, in_channels, kernel_size = 1, 1, 3
input_w, input_h = A.shape[2:]

output_w = (input_w - kernel_size) + 1
output_h = (input_h - kernel_size) + 1
Z = np.zeros((batch, out_channels, output_w, output_h), dtype=A.dtype)
print(Z.shape)
max_indexes = np.zeros(np.append(Z.shape, 2), dtype=np.int)
# print(max_indexes.shape)

for b in range(batch):
    for l in range(out_channels):
        for i in range(output_w):
            for j in range(output_h):
                print("_" * 34)
                window = A[b, l, i : i + kernel_size, j : j + kernel_size]
                print(window)
                
                x = np.unravel_index(np.argmax(window), window.shape)[0] + i
                y = np.unravel_index(np.argmax(window), window.shape)[1] + j
                max_indexes[b, l, i, j, :] = [x, y]
                Z[b, l, i, j] = A[b, l, x, y]

print(Z)
# print(Z.shape)


[[[[ 2  1  2 -2  1]
   [ 1 -2  2  3  1]
   [ 3 -2  1  0  0]
   [-2 -1 -2  2 -1]
   [-1  0  3  2 -1]]]]
(1, 1, 5, 5)
(1, 1, 3, 3)
__________________________________
[[ 2  1  2]
 [ 1 -2  2]
 [ 3 -2  1]]
__________________________________
[[ 1  2 -2]
 [-2  2  3]
 [-2  1  0]]
__________________________________
[[ 2 -2  1]
 [ 2  3  1]
 [ 1  0  0]]
__________________________________
[[ 1 -2  2]
 [ 3 -2  1]
 [-2 -1 -2]]
__________________________________
[[-2  2  3]
 [-2  1  0]
 [-1 -2  2]]
__________________________________
[[ 2  3  1]
 [ 1  0  0]
 [-2  2 -1]]
__________________________________
[[ 3 -2  1]
 [-2 -1 -2]
 [-1  0  3]]
__________________________________
[[-2  1  0]
 [-1 -2  2]
 [ 0  3  2]]
__________________________________
[[ 1  0  0]
 [-2  2 -1]
 [ 3  2 -1]]
[[[[3 3 3]
   [3 3 3]
   [3 3 3]]]]


In [ ]:
dZ_map = np.ones((2, 2, 3, 3)) * 9

dLdA = np.zeros_like(A, dtype=np.float)  # TODO
print(max_indexes.shape)
for b in range(batch):
    for l in range(out_channels):
        for i in range(output_w):
            for j in range(output_h):
                x, y = max_indexes[b, l, i, j]
                print(f"( {x}, {y})")
                dLdA[b, l, x, y] += dZ_map[b, l, i, j]

print(dLdA)

In [14]:
A = np.array([
    [
        [[2, 1, 2, -2, 1], [1, -2, 2, 3, 1], [3, -2, 1, 0, 0], [-2, -1, -2, 2, -1], [-1, 0, 3, 2, -1]],
        [[2, 1, 2, -2, 1], [1, -2, 2, 3, 1], [3, -2, 1, 0, 0], [-2, -1, -2, 2, -1], [-1, 0, 3, 2, -1]]
        ]
    ])
batch, in_channels, input_w, input_h = A.shape
out_channels = in_channels
kernel_size = 3
W = np.ones((out_channels, in_channels, kernel_size, kernel_size)) / kernel_size
W = np.repeat(W[np.newaxis, :, :, :, :], batch, axis=0)

output_w = (input_w - kernel_size) + 1
output_h = (input_h - kernel_size) + 1

Z = np.zeros((batch, out_channels, output_w, output_h), dtype=np.float)
W = np.repeat(W[np.newaxis, :, :, :, :], batch, axis=0)

for l in range(out_channels):
    for i in range(input_w - kernel_size + 1):
        for j in range(input_h - kernel_size + 1):
            # Z[b, l, i, j] = np.sum(A[b, l, i : i + kernel_size, j : j + kernel_size] * W[:, l, l, :, :]) / 3
            # print(np.sum(A[:, :, i : i + kernel_size, j : j + kernel_size] * W[:, l, :, :, :]))
            print(A[:, l, i : i + kernel_size, j : j + kernel_size] * W[:, l, :, :, :])
            Z[:, l, i, j] = (
                np.sum(A[:, :, i : i + kernel_size, j : j + kernel_size] * W[:, l, :, :, :]) / kernel_size
            )
            
print(Z)

[[[[[ 0.66666667  0.33333333  0.66666667]
    [ 0.33333333 -0.66666667  0.66666667]
    [ 1.         -0.66666667  0.33333333]]

   [[ 0.66666667  0.33333333  0.66666667]
    [ 0.33333333 -0.66666667  0.66666667]
    [ 1.         -0.66666667  0.33333333]]]


  [[[ 0.66666667  0.33333333  0.66666667]
    [ 0.33333333 -0.66666667  0.66666667]
    [ 1.         -0.66666667  0.33333333]]

   [[ 0.66666667  0.33333333  0.66666667]
    [ 0.33333333 -0.66666667  0.66666667]
    [ 1.         -0.66666667  0.33333333]]]]]
[[[[[ 0.33333333  0.66666667 -0.66666667]
    [-0.66666667  0.66666667  1.        ]
    [-0.66666667  0.33333333  0.        ]]

   [[ 0.33333333  0.66666667 -0.66666667]
    [-0.66666667  0.66666667  1.        ]
    [-0.66666667  0.33333333  0.        ]]]


  [[[ 0.33333333  0.66666667 -0.66666667]
    [-0.66666667  0.66666667  1.        ]
    [-0.66666667  0.33333333  0.        ]]

   [[ 0.33333333  0.66666667 -0.66666667]
    [-0.66666667  0.66666667  1.        ]
    [-0.666666

IndexError: index 1 is out of bounds for axis 1 with size 1

In [315]:
dZ_map = np.ones((1, 2, 3, 3)) * 9

dLdA = np.zeros_like(A, dtype=np.float)  # TODO
dZ_map_pad = np.pad(
    dZ_map,
    ((0, 0), (0, 0), (kernel_size - 1, kernel_size - 1), (kernel_size - 1, kernel_size - 1)),
    mode="constant",
)

W = np.repeat(W[np.newaxis, :, :, :, :], batch, axis=0)

for l in range(out_channels):
    for i in range(input_w):
        for j in range(input_h):
            dLdA[:, :, i, j] += np.sum(
                dZ_map_pad[:, l, i : i + kernel_size, j : j + kernel_size] * W[:, l, :, :, :]
            ) / kernel

print(dLdA)

[[[[1. 2. 3. 2. 1.]
   [2. 4. 6. 4. 2.]
   [3. 6. 9. 6. 3.]
   [2. 4. 6. 4. 2.]
   [1. 2. 3. 2. 1.]]]]


In [ ]:
import os, sys
import numpy as np
os.chdir(r"/mnt/d/CMU/Academics/Intro to DL/cmu-deeplearning/Homework 2/HW2P1")  #! Remove
sys.path.append("mytorch")
from pool import *
A = np.array([
    [
        [[2, 1, 2, -2, 1], [1, -2, 2, 3, 1], [3, -2, 1, 0, 0], [-2, -1, -2, 2, -1], [-1, 0, 3, 2, -1]],
        [[2, 1, 2, -2, 1], [1, -2, 2, 3, 1], [3, -2, 1, 0, 0], [-2, -1, -2, 2, -1], [-1, 0, 3, 2, -1]]
        ], 
            [
        [[2, 1, 2, -2, 1], [1, -2, 2, 3, 1], [3, -2, 1, 0, 0], [-2, -1, -2, 2, -1], [-1, 0, 3, 2, -1]],
        [[2, 1, 2, -2, 1], [1, -2, 2, 3, 1], [3, -2, 1, 0, 0], [-2, -1, -2, 2, -1], [-1, 0, 3, 2, -1]]
        ]
    ])
dLdZ = np.ones((2, 2, 3, 3)) * 9

m = MeanPool2d_stride1(3)
print(m.forward(A))
print(m.backward(dLdZ))

In [56]:
w = np.load(r'/mnt/d/CMU/Academics/Intro to DL/cmu-deeplearning/Homework 2/HW2P1/autograder/hw2_autograder/weights/mlp_weights_part_c.npy', allow_pickle=True)
res = np.load(r'/mnt/d/CMU/Academics/Intro to DL/cmu-deeplearning/Homework 2/HW2P1/autograder/hw2_autograder/ref_result/res_c.npy', allow_pickle=True)
print(w[1].shape)
print(res.shape)

(8, 16)
(1, 124)


In [89]:
np.set_printoptions(threshold=sys.maxsize)
a = w[0]
# print(a)
print(a.reshape(8, 24, 8).transpose([1, 2, 0]))

[[[ 1. -1. nan nan nan nan nan nan]
  [ 1. -1. nan nan nan nan nan nan]
  [nan nan  1. -1. nan nan nan nan]
  [nan nan  1. -1. nan nan nan nan]
  [nan nan nan nan  1. -1. nan nan]
  [nan nan nan nan  1. -1. nan nan]
  [nan nan nan nan nan nan  1. -1.]
  [nan nan nan nan nan nan  1. -1.]]

 [[-1.  1. nan nan nan nan nan nan]
  [-1. -1. nan nan nan nan nan nan]
  [nan nan -1.  1. nan nan nan nan]
  [nan nan -1. -1. nan nan nan nan]
  [nan nan nan nan -1.  1. nan nan]
  [nan nan nan nan -1. -1. nan nan]
  [nan nan nan nan nan nan -1.  1.]
  [nan nan nan nan nan nan -1. -1.]]

 [[-1.  1. nan nan nan nan nan nan]
  [-1.  1. nan nan nan nan nan nan]
  [nan nan -1.  1. nan nan nan nan]
  [nan nan -1.  1. nan nan nan nan]
  [nan nan nan nan -1.  1. nan nan]
  [nan nan nan nan -1.  1. nan nan]
  [nan nan nan nan nan nan -1.  1.]
  [nan nan nan nan nan nan -1.  1.]]

 [[-1. -1. nan nan nan nan nan nan]
  [ 1.  1. nan nan nan nan nan nan]
  [nan nan -1. -1. nan nan nan nan]
  [nan nan  1.  1. nan

In [53]:
w[1].flatten()

array([-1.,  1., -1.,  1., -1.,  1., -1.,  1., nan, nan, nan, nan, nan,
       nan, nan, nan,  1.,  1.,  1., -1., -1., -1., -1.,  1., nan, nan,
       nan, nan, nan, nan, nan, nan, -1.,  1.,  1., -1.,  1., -1., -1.,
        1., nan, nan, nan, nan, nan, nan, nan, nan, -1.,  1.,  1., -1.,
        1., -1., -1.,  1., nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, -1.,  1., -1.,  1., -1.,  1.,
       -1.,  1., nan, nan, nan, nan, nan, nan, nan, nan,  1.,  1.,  1.,
       -1., -1., -1., -1.,  1., nan, nan, nan, nan, nan, nan, nan, nan,
       -1.,  1.,  1., -1.,  1., -1., -1.,  1., nan, nan, nan, nan, nan,
       nan, nan, nan, -1.,  1.,  1., -1.,  1., -1., -1.,  1.])